In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from autocorrect import Speller

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

import findspark
findspark.init()

import pyspark
import pyspark.sql
from pyspark.sql import *
from pyspark.sql.functions import *

# Dataset(s) preparation and cleaning

Before we proceed to tackle each of our research questions, some data cleaning is in order.

## Load the data and explore its structure

In [3]:
inspections = pd.read_csv('datasets/food-inspections.csv')

In [4]:
len(inspections)

195736

The dataset has 22 columns. Let's examine what each of them is.

In [5]:
#Display columns
inspections.columns

Index(['Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Facility Type',
       'Risk', 'Address', 'City', 'State', 'Zip', 'Inspection Date',
       'Inspection Type', 'Results', 'Violations', 'Latitude', 'Longitude',
       'Location', 'Historical Wards 2003-2015', 'Zip Codes',
       'Community Areas', 'Census Tracts', 'Wards'],
      dtype='object')

## Clean data

The 'Location' column contains the latitude and longitude of the establishment. However, there are separate 'Latitude' and 'Longitude' columns. We can hence safely drop the 'Location' column.

In [6]:
# inspections = inspections.drop(columns=['Location'])

The head of the dataset only contains NaN entries for the 'Historical Wards 2003-2015', 'Zip Codes', 'Community Areas', 'Census Tracts', 'Wards' columns. Let's see if this is true for the whole dataset.

In [7]:
# make sure that our assumption is correct
print('Values taken by \'Historical Wards 2003-2015\': ', inspections['Zip Codes'].unique())
print('Values taken by \'Zip Codes\': ', inspections['Zip Codes'].unique())
print('Values taken by \'Community Areas\': ', inspections['Zip Codes'].unique())
print('Values taken by \'Census Tracts\': ', inspections['Zip Codes'].unique())
print('Values taken by \'Wards\': ', inspections['Zip Codes'].unique())


Values taken by 'Historical Wards 2003-2015':  [nan]
Values taken by 'Zip Codes':  [nan]
Values taken by 'Community Areas':  [nan]
Values taken by 'Census Tracts':  [nan]
Values taken by 'Wards':  [nan]


We drop all columns apart from the 'Community Areas' because we will be needing it in our study. We will fill later.

In [8]:
inspections = inspections.drop(columns=['Historical Wards 2003-2015'])
inspections = inspections.drop(columns=['Zip Codes'])
inspections = inspections.drop(columns=['Census Tracts'])
inspections = inspections.drop(columns=['Wards'])

Let's examine if the whole dataset is relevent to the study we are conducting by seeing which entries correspond to facilities in Chicago.

First, we check if there are any missing values for the column 'City' or 'State'

In [9]:
#Investigate the state=nan and city=nan restaurants
inspections[pd.isnull(inspections.State) | pd.isnull(inspections.City)]

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Community Areas
1669,2312774,CHICAGO COLLEGIATE CHARTER,CHICAGO COLLEGIATE CHARTER,3846104.0,School,Risk 1 (High),10909 S COTTAGE GROVE AVE,NaN,IL,NaN,2019-09-24T00:00:00.000,Canvass Re-Inspection,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.696087,-87.608945,"{'longitude': '41.696086647178035', 'latitude'...",NaN
1879,2312540,CHICAGO COLLEGIATE CHARTER,CHICAGO COLLEGIATE CHARTER,3846104.0,School,Risk 1 (High),10909 S COTTAGE GROVE AVE,NaN,IL,NaN,2019-09-19T00:00:00.000,Canvass Re-Inspection,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.696087,-87.608945,"{'longitude': '41.696086647178035', 'latitude'...",NaN
1903,2312545,JCYS IRIS & STEVEN PODOLSKY FAMILY CENTER,JCYS IRIS & STEVEN PODOLSKY FAMILY CENTER,2671297.0,Children's Services Facility,Risk 1 (High),2112 W LAWRENCE AVE,NaN,IL,60625.0,2019-09-19T00:00:00.000,License Re-Inspection,Pass,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.968821,-87.682201,"{'longitude': '41.968821253748864', 'latitude'...",NaN
3073,2305166,"AMY BECK CAKE DESIGN, LLC","AMY BECK CAKE DESIGN, LLC",2079264.0,Bakery,Risk 1 (High),636 N RACINE AVE,NaN,NaN,60642.0,2019-08-23T00:00:00.000,Canvass,Pass,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...",41.893380,-87.657588,"{'longitude': '41.893380429024546', 'latitude'...",NaN
3617,2304583,JCYS IRIS & STEVEN PODOLSKY FAMILY CENTER,JCYS IRIS & STEVEN PODOLSKY FAMILY CENTER,2671297.0,Children's Services Facility,Risk 1 (High),2112 W LAWRENCE AVE,NaN,IL,60625.0,2019-08-13T00:00:00.000,License,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.968821,-87.682201,"{'longitude': '41.968821253748864', 'latitude'...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194253,60291,"CLOVERHILL PASTRY-VEND,LLC","CLOVERHILL PASTRY-VEND,LLC",2004357.0,Wholesale,Risk 3 (Low),4464 W 44TH ST,NaN,IL,60632.0,2010-02-03T00:00:00.000,License Re-Inspection,Pass,NaN,41.814266,-87.736013,"{'longitude': '41.81426627941673', 'latitude':...",NaN
194489,60282,"CLOVERHILL PASTRY-VEND,LLC","CLOVERHILL PASTRY-VEND,LLC",2004357.0,Wholesale,Risk 3 (Low),4464 W 44TH ST,NaN,IL,60632.0,2010-01-28T00:00:00.000,License,Fail,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.814266,-87.736013,"{'longitude': '41.81426627941673', 'latitude':...",NaN
194610,60279,"CLOVERHILL PASTRY-VEND,LLC","CLOVERHILL PASTRY-VEND,LLC",2004357.0,Wholesale,Risk 3 (Low),4464 W 44TH ST,NaN,IL,60632.0,2010-01-27T00:00:00.000,License,Fail,NaN,41.814266,-87.736013,"{'longitude': '41.81426627941673', 'latitude':...",NaN
195141,67912,THREE CHEFS RESTURANT,THREE CHEFS RESTURANT,2009471.0,Restaurant,Risk 1 (High),8125 S HALSTED ST,NaN,IL,60620.0,2010-01-15T00:00:00.000,License Re-Inspection,Pass,NaN,41.746236,-87.643766,"{'longitude': '41.74623627171974', 'latitude':...",NaN


Looking at the coordinates of these places, all of them seem to also be in chicago, so we will fill their City and State columns

In [10]:
inspections['City'] = inspections['City'].fillna('Chicago')
inspections['State'] = inspections['State'].fillna('IL')

Next, we check if there are any facilities which are not located in Chicago.

In [11]:
# make sure that our assumption is correct
print('Values taken by \'City\': ', inspections['City'].unique())

Values taken by 'City':  ['CHICAGO' 'Chicago' 'chicago' 'GRIFFITH' 'NEW YORK' 'SCHAUMBURG'
 'ELMHURST' 'ALGONQUIN' 'NEW HOLSTEIN' 'CCHICAGO' 'NILES NILES' 'EVANSTON'
 'CHICAGO.' 'CHESTNUT STREET' 'LANSING' 'CHICAGOCHICAGO' 'WADSWORTH'
 'WILMETTE' 'WHEATON' 'CHICAGOHICAGO' 'ROSEMONT' 'CHicago' 'CALUMET CITY'
 'PLAINFIELD' 'HIGHLAND PARK' 'PALOS PARK' 'ELK GROVE VILLAGE' 'CICERO'
 'BRIDGEVIEW' 'OAK PARK' 'MAYWOOD' 'LAKE BLUFF' '312CHICAGO'
 'SCHILLER PARK' 'SKOKIE' 'BEDFORD PARK' 'BANNOCKBURNDEERFIELD' 'CHCICAGO'
 'BLOOMINGDALE' 'Norridge' 'CHARLES A HAYES' 'CHCHICAGO' 'CHICAGOI'
 'SUMMIT' 'OOLYMPIA FIELDS' 'WESTMONT' 'CHICAGO HEIGHTS' 'JUSTICE'
 'TINLEY PARK' 'LOMBARD' 'EAST HAZEL CREST' 'COUNTRY CLUB HILLS'
 'STREAMWOOD' 'BOLINGBROOK' 'INACTIVE' 'BERWYN' 'BURNHAM' 'DES PLAINES'
 'LAKE ZURICH' 'OLYMPIA FIELDS' 'alsip' 'OAK LAWN' 'BLUE ISLAND' 'GLENCOE'
 'FRANKFORT' 'NAPERVILLE' 'BROADVIEW' 'WORTH' 'Maywood' 'ALSIP'
 'EVERGREEN PARK']


We can see that this column takes values which are not Chicago. The rows where the 'City' is not Chicago are hence irrelevent to our study and should be dropped. Let's first make sure tha the bulk of the data is for Chicago before proceeding

In [12]:
chicago_inspections = inspections.groupby('City')['Inspection ID'].nunique().filter(regex='(?i)chicago', axis=0)
print('{}% of the inpections in the dataframe come from Chicago.'.format(100 * chicago_inspections.values.sum()/len(inspections)))

99.7884906200188% of the inpections in the dataframe come from Chicago.


We can safely drop the rows which come from cities that are not Chicago.

In [13]:
# list of ways Chicago has been written in the dataset
chicago_variations = chicago_inspections.index.tolist()
inspections = inspections[inspections['City'].isin(chicago_variations)]
# drop the 'City' and 'State' columns since they have each only one value, 'Chicago' and 'IL' respectively
inspections = inspections.drop(columns=['City', 'State'])


Now that we only have facilities in Chicago in our dataset, let us fill the 'Community Areas' column. To that end, we use the geopy library.

In [14]:
# def getareanneighbourhood(coord):
#     """
    
#     """
#     geolocator = Nominatim(timeout=10,user_agent="area_filler")
#     geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#     dic = geocode.reverse(coord).raw['address']
#     return dic.get('suburb', np.nan), dic.get('neighbourhood', np.nan)

def combineloc(latitude, longitude):
    """
    function to format the latitude and longitude such that they can be used in geopy requests
    """
    return '{}, {}'.format(latitude, longitude)

In [15]:
locations = inspections['Latitude'].dropna().combine(inspections['Longitude'].dropna(),combineloc)
unique_locs = locations.unique()

In [16]:
unique_locs

array(['41.92799528871574, -87.78575236468352',
       '41.946140053442825, -87.73518301995274',
       '41.93592957402078, -87.64440716256712', ...,
       '41.764896400247046, -87.65396483351302',
       '41.768328334800714, -87.67381938402686',
       '41.846516428599394, -87.69542345938575'], dtype=object)

In [17]:
len(unique_locs)

16812

In [18]:
unique_locs_s = pd.Series(unique_locs, dtype=str)

We then request the geopy entry for the locations we have (code takes 4h40 to run as we can only do one geopy query per second) and save the areas in a pickle.

In [19]:
# geolocator = Nominatim(timeout=17000,user_agent="area_filler")
# geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)
# # for i in unique_locs:
# #     print(i)
# #     print(geolocator.reverse(i))
# areas = unique_locs_s.copy().apply(geocode)
# areas.to_pickle('./areas')

In [20]:
areas = pd.read_pickle('./areas.pickle')

In [21]:
areas.isna().sum()

0

Let's add the community areas and neighborhoods to the dataframe.

In [22]:
# get latitude, longitude and corresponding community area and neighbourhood in same dataframe
suburbs_neighbourhoods = [(x.raw.get('address', {}).get('suburb',np.nan), x.raw.get('address', {}).get('neighbourhood',np.nan)) for x in areas]
suburbs, neighbourhoods = zip(*suburbs_neighbourhoods)
locs_df = pd.concat([pd.Series(unique_locs, name='Location'), pd.Series(suburbs,name='Community Area'), pd.Series(neighbourhoods,name='Neighbourhood')], axis=1)

In [23]:
# add the community area and the neighbourhood to each entry in our dataframe
inspections['Location'] = inspections['Latitude'].combine(inspections['Longitude'],combineloc)
inspections = inspections.merge(locs_df,on='Location',how='outer')
inspections = inspections.drop(columns=['Community Areas'])

Let's check if there are any NaN entries in our 'Community Area' column

In [24]:
print('{}% of rows don\'t have missing Community Areas'.format(100 * (1 - inspections['Community Area'].isna().sum()/len(inspections))))

96.80873914511031% of rows don't have missing Community Areas


We may safely drop the rows which have null 'Community Area'.

In [25]:
inspections = inspections[inspections['Community Area'].notna()]

Let's check if there are anymore missing values in the dataframe.

In [26]:
inspections.isna().sum().apply(lambda x: '{}% missing values'.format(100 * x/len(inspections)))

Inspection ID                        0.0% missing values
DBA Name                             0.0% missing values
AKA Name               1.254662060075861% missing values
License #           0.008980738956332477% missing values
Facility Type          2.453326571365178% missing values
Risk                 0.03592295582532991% missing values
Address                              0.0% missing values
Zip                 0.024829101820448615% missing values
Inspection Date                      0.0% missing values
Inspection Type    0.0005282787621372046% missing values
Results                              0.0% missing values
Violations            26.585628704554818% missing values
Latitude                             0.0% missing values
Longitude                            0.0% missing values
Location                             0.0% missing values
Community Area                       0.0% missing values
Neighbourhood         16.824622016545693% missing values
dtype: object

* The AKA names still have missing entries. We will leave them as they are but will be hence mostly sticking to the DBA Name when referring to establishments.
* The Lisence Number is missing for some entries. Seeing as it is not essential in our main analysis we will not pay attention to it for now.
* The missing Zip entries are not important as we have enough information regarding location (latitude, longitude, community area and address)
* The number of missing neighbourhoods is quite big. Hence, we may drop that column.
* The missing facility type, risk, inspection type and violations entries might hinder our analysis and hence we focus on cleaning those column next.

In [27]:
# drop neighbourhood column
inspections = inspections.drop(columns=['Neighbourhood'])

We first examine the facility type entries

In [28]:
# from nltk.corpus import stopwords
# stop = stopwords.words('english')
spell = Speller(lang='en')
inspections['Facility Type'] = inspections['Facility Type'].str.lower().apply(lambda x: spell(str(x)))
unique_types = pd.Series(inspections['Facility Type'].unique())
unique_types = pd.Series(unique_types.apply(lambda x: re.sub(',|&|;','/',str(x)).split('/')).explode().unique())
unique_types.apply(lambda x: str(x).strip())

0                        restaurant
1                               nan
2                     grocery store
3      children's services facility
4                       coffee shop
                   ...             
375                    soup kitchen
376                    hooka lounge
377                       religious
378                wholesale bakery
379                      kids cafe'
Length: 380, dtype: object

In [29]:
#corrected_unique_types = unique_types.apply(lambda x: spell(str(x).strip()))

In [30]:
#corrected_unique_types

There are 427 facility types in the dataframe. Let's see if we can cluster some of them.

In [31]:
school_vocab = ['school', 'under 6', 'university', 'cafeteria', 'training program', 'kids', 'children', 'daycare', 'years', 'school cafeteria', 'college', 'class', 'day care']
restaurant_vocab = ['restaurant', 'smokehouse', 'diner', 'breakfast', 'lunch', 'grill', 'sushi','banquet', 'dining', 'taqueria']
homes_vocab = ['long term care','assisted living', 'nursing', 'care', 'supportive']

religious_vocab = ['religious', 'religion', 'church', 'synaguogue', 'temple']
coffee_vocab = ['coffee', 'cafe']
catering_vocab = ['cater']

bakery_vocab = ['bake', 'patisserie', 'boulangerie']
market_vocab = ['butcher', 'snack', 'dollar', 'grocery', 'frozen food storage', 'meat packing', 'food', 'market', 'packaged', 'popcorn', 'pantry', 'store', 'produce', 'kiosk', 'convenience', 'commiasary','gas station', 'wholesale', 'deli', 'convenient', 'retail', 'dollar tree']
dessert_vocab = ['ice cream', 'dessert', 'paleteria', 'candy', 'gelato', 'donut']

health_vocab = ['gym', 'exercise', 'nutrition','herbal', 'fitness', 'drug', 'rehab', 'herbalife', 'weight', 'herb', 'health', 'decream']
medical_vocab = ['pharmacy']
vending_vocab = ['vend', 'pop-up', 'mobile', 'cart', 'dispenser', 'truck']

drinks_vocab = ['liquor', 'music', 'bar', 'pub', 'beverage', 'club', 'roof', 'tavern', 'brewery', 'wine', 'beer', 'lounge', 'tea', 'shakes']
live_vocab =['live', 'poultry', 'slaughter', 'farm']
kitchen_vocab = ['kitchen']

distributor_vocab = ['distributor', 'distribution']
shelter_vocab = ['shelter', 'youth housing']
# other_vocab = ['warehouse', 'other', 'theater', 'golf', 'laundromat', 'hotel', 'riverbank', 'theatre', 'event', 'special', 'museum', 'hospital', 'pool', 'art', 'airport', 'gallery', 'terminal']


In [32]:
#unique_types = corrected_unique_types

In [33]:
#'live poultry' in corrected_unique_types.values

In [34]:
def extract_label(unique_labels, vocab,mssg):
    labels = unique_types.apply(lambda x: x if bool(re.search('|'.join(vocab), x)) else np.nan).dropna()
    print(labels)
    print('{}: {}'.format(mssg, labels.size))
    unique_labels = unique_types.drop(labels=labels.index,errors='ignore')
    return unique_labels, labels

In [35]:
unique_types, school_labels = extract_label(unique_types, school_vocab, 'Number of Schools: ')
unique_types, restaurant_labels = extract_label(unique_types, restaurant_vocab, 'Number of Restaurants: ')
unique_types, homes_labels = extract_label(unique_types, homes_vocab, 'Number of Nursing homes: ')

unique_types, religious_labels = extract_label(unique_types, religious_vocab, 'Number of Religious Establishments: ')
unique_types, coffee_labels = extract_label(unique_types, coffee_vocab, 'Number of Coffeeshops: ')
unique_types, catering_labels = extract_label(unique_types, catering_vocab, 'Number of Caterers: ')

unique_types, bakery_labels = extract_label(unique_types, bakery_vocab, 'Number of Bakeries: ')
unique_types, market_labels = extract_label(unique_types, market_vocab, 'Number of Markets: ')
unique_types, dessert_labels = extract_label(unique_types, dessert_vocab, 'Number of Dessert Places: ')

unique_types, health_labels = extract_label(unique_types, health_vocab, 'Number of Health Institutions: ')
unique_types, medical_labels = extract_label(unique_types, medical_vocab, 'Number of Pharmacies: ')
unique_types, vending_labels = extract_label(unique_types, vending_vocab, 'Number of Vending Establishments: ')

unique_types, drinks_labels = extract_label(unique_types, drinks_vocab, 'Number of Drinks Places: ')
unique_types, live_labels = extract_label(unique_types, live_vocab, 'Number of Live Animal Sellers and Slaughterhouses: ')
unique_types, kitchen_labels = extract_label(unique_types, kitchen_vocab, 'Number of Shared Kitchen: ')

unique_types, distributor_labels = extract_label(unique_types, distributor_vocab, 'Number of Distributors: ')
unique_types, shelter_labels = extract_label(unique_types, shelter_vocab, 'Number of Shelters: ')
other_labels = unique_types

3                children's services facility
7                       daycare (2 - 6 years)
8             daycare above and under 2 years
20                                     school
22                   15 months to 5 years old
23                         daycare combo 1586
29                    daycare (under 2 years)
31                      day care combo (1586)
32                             charter school
33         1023 children's service s facility
42                            teaching school
72                             cooking school
73                             private school
77                          daycare (2 years)
79                                  cafeteria
83                              daycare night
84                       after school program
91                                    daycare
112                           senior day care
121         1023-children's services facility
129                           culinary school
132         1023 children's servic

In [36]:
unique_types[0:10]

1                      nan
15    silverware warehouse
47                 theater
48           special event
50             psalteteria
58               incubator
62                hospital
63                  museum
64                 gallery
66                    pool
dtype: object

**Explore the difference between DBA and AKA names**

In [26]:
print ('There are {0} unique DBA (‘Doing business as.’) names in the dataset.'.format(len(inspections['DBA Name'].unique())))

There are 26656 unique DBA (‘Doing business as.’) names in the dataset.


In [27]:
# Display the number of restaurants (we display the unique DBA names)
print ('There are {0} AKA (‘Also known as.’) names in the dataset.'.format(len(inspections['AKA Name'].unique())))

There are 25434 AKA (‘Also known as.’) names in the dataset.


In [28]:
# Explore how DBA and AKA names differ
print ('There are {0} rows where the DBA names and the AKA names differ.'\
       .format((len(inspections[inspections['DBA Name'] != inspections['AKA Name']]))))

There are 49418 rows where the DBA names and the AKA names differ.


In [29]:
print('Examples of different DBA and AKA names : ')
inspections[inspections['DBA Name'] != inspections['AKA Name']].head(2)

Examples of different DBA and AKA names : 


,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Community Area,Neighbourhood
8,2059948,Subway Sandwiches,Subway,1621730.0,Restaurant,Risk 1 (High),2620 N NARRAGANSETT AVE,60639.0,2017-06-09T00:00:00.000,Canvass Re-Inspection,Pass,NaN,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",Belmont Cragin,Beat 2512
10,2059950,Subway Sandwiches,Subway,1621730.0,Restaurant,Risk 1 (High),2620 N NARRAGANSETT AVE,60639.0,2017-06-09T00:00:00.000,Canvass,Out of Business,NaN,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",Belmont Cragin,Beat 2512
